In [63]:
from keras.models import Sequential
from keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from keras.callbacks import EarlyStopping
from PIL import Image

In [64]:
# ilkleme
classifier = Sequential()

# Adım 1 - Convolution

classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))

# Adım 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# 2. convolution katmanı
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adım 3 - Flattening
classifier.add(Flatten())

# Adım 4 - YSA
classifier.add(Dense(128, activation = 'relu'))
classifier.add(Dense(1, activation = 'sigmoid'))

In [65]:
# CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [66]:
# CNN ve resimler
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('data/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 1,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('data/test_set',
                                            target_size = (64, 64),
                                            batch_size = 1,
                                            class_mode = 'binary')


Found 35 images belonging to 2 classes.
Found 17 images belonging to 2 classes.


In [67]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 2000,
                         epochs = 1000,
                         validation_data = test_set,
                         validation_steps = 2000,
                        verbose = 1)

Epoch 1/1000


<ipython-input-67-0f9963715c65>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(training_set,


2000/2000 [==============================] - 2s 909us/step - loss: 0.6795 - accuracy: 0.7143 - val_loss: 0.5961 - val_accuracy: 0.7059


In [68]:
earlyStopping = EarlyStopping(monitor="val_loss", mode="min",verbose=1,patience = 5)

In [69]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 2000,
                         epochs = 1000,
                         validation_data = test_set,
                         validation_steps = 2000,
                        verbose = 1,
                        callbacks = [earlyStopping])

Epoch 1/1000
   3/2000 [..............................] - ETA: 1:33 - loss: 0.4070 - accuracy: 1.0000

<ipython-input-69-eba3e4b83f84>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(training_set,


2000/2000 [==============================] - 2s 911us/step - loss: 0.6058 - accuracy: 0.7143 - val_loss: 0.6001 - val_accuracy: 0.7059


In [70]:
import numpy as np
import pandas as pd

In [71]:
test_set.reset()
pred=classifier.predict_generator(test_set,verbose=1)
#pred = list(map(round,pred))
pred[pred > .5] = 1
pred[pred <= .5] = 0

 5/17 [=======>......................] - ETA: 0s

<ipython-input-71-f223e6cf5bc6>:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred=classifier.predict_generator(test_set,verbose=1)


17/17 [==============================] - 0s 20ms/step


In [72]:
#labels = (training_set.class_indices)

test_labels = []

for i in range(0,int(17)):
    test_labels.extend(np.array(test_set[i][1]))
    print(test_set[i][1])
print('test_labels')
print(test_labels)

[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
test_labels
[1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0]


In [73]:
dosyaisimleri = test_set.filenames
#abc = test_set.
#print(idx)
#test_labels = test_set.
sonuc = pd.DataFrame()
sonuc['dosyaisimleri']= dosyaisimleri
sonuc['tahminler'] = pred
sonuc['test'] = test_labels   

In [74]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_labels, pred)
print (cm)

[[ 0  5]
 [ 0 12]]


In [4]:
classifier.save("kerasModel")

NameError: name 'classifier' is not defined